In [10]:
import numpy as np
import matplotlib.pyplot as plt
from pyqtgraph.Qt import QtGui
import pyqtgraph as pg

from brian2 import Network, core, prefs, defaultclock, float64
from brian2.units import *
from brian2 import PoissonGroup, SpikeGeneratorGroup
from brian2 import SpikeMonitor, StateMonitor

from teili import Neurons, Connections, TeiliNetwork
from teili.models.neuron_models import ExpLIF as neuron_model
from teili.models.synapse_models import Alpha as synapse_model
from teili.stimuli.testbench import SequenceTestbench, OCTA_Testbench

from teili.tools.visualizer.DataViewers import PlotSettings
from teili.tools.visualizer.DataControllers import Rasterplot, Lineplot

prefs.codegen.target = "numpy"
defaultclock.dt = 0.1 * ms
core.default_float_dtype = float64

In [35]:
n_neurons = 80
rate_testbench = False
weight = 0.5
duration = 2

connectivity_probabiliy={"pyramidal_pyramidal": 0.6, #old 0.4
                         "pyramidal_pv": 0.6, #old 0.5
                         "pv_pyramidal": 1.0,
                         "pv_pv": 0.6, #old 1.0
                         "thal_pyramidal": 0.4, # this needs checking old0.4
#                          "thal_pv": 0.5,
#                          "pyramidal_sst": 0.5,
#                          "pyramidal_vip": 0.5,
#                          "sst_pyramidal": 1.0,
#                          "sst_sst": 0.0,
#                          "sst_pv": 0.86,
#                          "sst_vip": 0.9,
#                          "pv_sst": 0.0,
#                          "pv_vip": 0.0,
#                          "vip_pyramidal": 0.15,
#                          "vip_sst": 0.63,
#                          "vip_pv": 0.0,
#                          "vip_vip": 0.0,
#                          "thal_sst": 0.5,
#                          "thal_vip": 0.5
                        }

ctx = TeiliNetwork()

# load sequence into SpikeGenerator, set the period
if not rate_testbench:
    tb = OCTA_Testbench()
    thal = tb.rotating_bar(length=10, 
                           nrows=10,
                           direction='cw',
                           ts_offset=3, 
                           angle_step=10,
                           noise_probability=0.2,
                           repetitions=300,
                           debug=False)
else:
    testbench = SequenceTestbench(n_channels=100, 
                                  n_items=7, 
                                  cycle_length=2000, 
                                  rate=50, 
                                  noise_probability=0.005)
    testbench.stimuli()
      
    thal = SpikeGeneratorGroup(N=np.max(testbench.indices)+1, 
                               indices=testbench.indices,
                               times=testbench.times*ms,
                               period=2000*ms)
    
# Create neural ensembles
pyramidal_neurons = Neurons(N=np.int(0.8 * n_neurons), 
                            equation_builder=neuron_model(num_inputs=3),
                            name='pyramidal_neurons')

pv_neurons = Neurons(N=np.int(0.2 * n_neurons),
                     equation_builder=neuron_model(num_inputs=2),
                     name='pv_neurons')

# Create synaptic connection among input, excitatory and inhibitory ensembles
thal_pyramidal = Connections(thal, pyramidal_neurons,
                       equation_builder=synapse_model(),
                       method='euler',
                       name='thal_pyramidal')
pyramidal_pyramidal = Connections(pyramidal_neurons, pyramidal_neurons,
                     equation_builder=synapse_model(),
                     method='euler',
                     name='pyramidal_pyramidal')
pyramidal_pv = Connections(pyramidal_neurons, pv_neurons,
                     equation_builder=synapse_model(),
                     method='euler',
                     name='pyramidal_pv')

pv_pyramidal = Connections(pv_neurons, pyramidal_neurons,
                     equation_builder=synapse_model(),
                     method='euler',
                     name='pv_pyramidal')

pv_pv = Connections(pv_neurons, pv_neurons,
                     equation_builder=synapse_model(),
                     method='euler',
                     name='pv_pv')

# Connect synaptic partners with probability of connection (p)
thal_pyramidal.connect(p=connectivity_probabiliy['thal_pyramidal'])
pyramidal_pyramidal.connect(p=connectivity_probabiliy['pyramidal_pyramidal'])
pyramidal_pv.connect(p=connectivity_probabiliy['pyramidal_pv'])
pv_pyramidal.connect(p=connectivity_probabiliy['pv_pyramidal'])
pv_pv.connect(p=connectivity_probabiliy['pv_pv'])

# Set inhibitory weights
thal_pyramidal.weight = 0.25
pyramidal_pyramidal.weight = weight
pyramidal_pv.weight = weight
pv_pyramidal.weight = -weight*2
pv_pv.weight = -weight*2

# Introducing hetereogenity in the network's parameter
# pyramidal_neurons.add_mismatch()
# pv_neurons.add_mismatch()
# thal_pyramidal.add_mismatch()
# pyramidal_pyramidal.add_mismatch()
# pv_pv.add_mismatch()
# pyramidal_pv.add_mismatch()
# pv_pyramidal.add_mismatch()

# Create SpikeMonitors to visualise the networks activity
thal_spike = SpikeMonitor(thal)
pyr_spike = SpikeMonitor(pyramidal_neurons)
pyr_vm = StateMonitor(pyramidal_neurons, variables='Vm', record=True)
pv_spike = SpikeMonitor(pv_neurons)
pv_vm = StateMonitor(pv_neurons, variables='Vm', record=True)

ctx.add(pyramidal_neurons, pv_neurons,
        pyramidal_pyramidal, pyramidal_pv,
        pv_pv, pv_pyramidal,
        thal, thal_pyramidal,
        pyr_spike, pv_spike, thal_spike,
        pyr_vm, pv_vm)

ctx.run(duration=duration*second, report='text')


Starting simulation at t=0. s for a duration of 2. s
1.1795 (58%) simulated in 10s, estimated 7s remaining.
2.0 (100%) simulated in 16s


In [36]:
# Visualize simulation results

def update():
    region.setZValue(10)
    minX, maxX = region.getRegion()
    p1.setXRange(minX, maxX, padding=0)
    p2.setXRange(minX, maxX, padding=0)
    p3.setXRange(minX, maxX, padding=0)
    p4.setXRange(minX, maxX, padding=0)


def updateRegion(window, viewRange):
    rgn = viewRange[0]
    region.setRegion(rgn)


app = QtGui.QApplication.instance()
if app is None:
    app = QtGui.QApplication(sys.argv)
else:
    print('QApplication instance already exists: %s' % str(app))

pg.setConfigOptions(antialias=True)
labelStyle = {'color': '#FFF', 'font-size': 12}
MyPlotSettings = PlotSettings(fontsize_title=labelStyle['font-size'],
                              fontsize_legend=labelStyle['font-size'],
                              fontsize_axis_labels=10,
                              marker_size=7)

win = pg.GraphicsWindow()
win.resize(2100, 1200)
win.setWindowTitle('Simple EI Network')

p0 = win.addPlot(title="Input spike raster plot", 
                 colspan=2)
win.nextRow()
p1 = win.addPlot(title="Excitatory membrane potential")
p2 = win.addPlot(title="Inhibitory membrane potential")
win.nextRow()
p3 = win.addPlot(title="Excitatory spike raster plot")
p4 = win.addPlot(title="Inhibitory spike raster plot")

p0.showGrid(x=True, y=True)
p1.showGrid(x=True, y=True)
p2.showGrid(x=True, y=True)
p3.showGrid(x=True, y=True)
p4.showGrid(x=True, y=True)

region = pg.LinearRegionItem()
region.setZValue(10)

p0.addItem(region, ignoreBounds=True)

Rasterplot(MyEventsModels=[thal_spike],
           MyPlotSettings=MyPlotSettings,
           time_range=[0, duration],
           neuron_id_range=None,
           title="Rasterplot of thalamus",
           xlabel='Time (ms)',
           ylabel="Neuron ID",
           backend='pyqtgraph',
           mainfig=win,
           subfig_rasterplot=p0,
           QtApp=app,
           show_immediately=False)


Lineplot(DataModel_to_x_and_y_attr=[(pyr_vm, ('t', 'Vm'))],
         MyPlotSettings=MyPlotSettings,
         x_range=[0, duration],
         title="Membrane potential Pyramidal cells",
         xlabel="Time (ms)",
         ylabel="Vm (mV)",
         backend='pyqtgraph',
         mainfig=win,
         subfig=p1,
         QtApp=app,
         show_immediately=False)

Lineplot(DataModel_to_x_and_y_attr=[(pv_vm, ('t', 'Vm'))],
         MyPlotSettings=MyPlotSettings,
         x_range=[0, duration],
         title="Membrane potential PV cells",
         xlabel="Time (ms)",
         ylabel="Vm (mV)",
         backend='pyqtgraph',
         mainfig=win,
         subfig=p2,
         QtApp=app,
         show_immediately=False)

Rasterplot(MyEventsModels=[pyr_spike],
           MyPlotSettings=MyPlotSettings,
           time_range=[0, duration],
           neuron_id_range=None,
           title="Rasterplot of pyramidal cells",
           xlabel='Time (ms)',
           ylabel="Neuron ID",
           backend='pyqtgraph',
           mainfig=win,
           subfig_rasterplot=p3,
           QtApp=app,
           show_immediately=False)

Rasterplot(MyEventsModels=[pv_spike],
           MyPlotSettings=MyPlotSettings,
           time_range=[0, duration],
           neuron_id_range=None,
           title="Rasterplot of PV cells",
           xlabel='Time (ms)',
           ylabel="Neuron ID",
           backend='pyqtgraph',
           mainfig=win,
           subfig_rasterplot=p4,
           QtApp=app,
           show_immediately=False)

region.sigRegionChanged.connect(update)
p1.sigRangeChanged.connect(updateRegion)
p2.sigRangeChanged.connect(updateRegion)
p3.sigRangeChanged.connect(updateRegion)
p4.sigRangeChanged.connect(updateRegion)

region.setRegion([1, 1.1])
p0.setXRange(0, duration, padding=0)
                     
app.exec()

QApplication instance already exists: <PyQt5.QtWidgets.QApplication object at 0x7f37be15fc30>


0